In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
INPUT_IMAGE_PATH = 'YOUR INPUT IMAGE PATH'
OUTPUT_IMAGE_PATH = 'YOUR OUTPUT IMAGE PATH'
HISTOGRAM_FILE_PATH = 'YOUR HISTOGRAM FILE PATH'
OUTPUT_TXT_PATH = 'YOUR OUTPUT TXT PATH'

In [ ]:
def load_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise FileNotFoundError(f"Failed: {image_path}")
    return image

In [ ]:
def hough_transform(binary_image):
    """excalculate Hough transform"""
    return cv2.HoughLinesP(binary_image, 5, np.pi / 3600, 1, minLineLength=1, maxLineGap=1)

In [ ]:
def calculate_angle_distribution(lines):
    """calculate angle distribution and pixel distribution"""
    angle_counts = {}
    pixel_distribution = {}

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
            angle = (angle + 180) % 180 
            angle = int(angle)

            if angle not in angle_counts:
                angle_counts[angle] = 0
                pixel_distribution[angle] = []

            angle_counts[angle] += 1
            pixels = bresenham(x1, y1, x2, y2)
            pixel_distribution[angle].extend(pixels)

    return angle_counts, pixel_distribution

def bresenham(x1, y1, x2, y2):
    """use Bresenham's line algorithm to get the pixel coordinates of a line"""
    pixels = []
    dx, dy = abs(x2 - x1), abs(y2 - y1)
    sx, sy = (1 if x1 < x2 else -1), (1 if y1 < y2 else -1)
    err = dx - dy

    while True:
        pixels.append((x1, y1))
        if x1 == x2 and y1 == y2:
            break
        e2 = err * 2
        if e2 > -dy:
            err -= dy
            x1 += sx
        if e2 < dx:
            err += dx
            y1 += sy

    return pixels

In [ ]:
def draw_lines(binary_image, lines):
    """draw lines on a blank image"""
    combined_image = np.zeros_like(binary_image)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(combined_image, (x1, y1), (x2, y2), 255, 1)
    return combined_image

def save_histogram(angle_counts, pixel_distribution, save_path):
    """draw histogram of angle distribution and save it"""
    angles = list(angle_counts.keys())
    pixel_counts = [len(pixel_distribution.get(angle, [])) for angle in angles]
    total_pixels = sum(pixel_counts)
    normalized_distribution = [count / total_pixels for count in pixel_counts]

    # calculate the percentage of pixels in the range of 86-91 degrees
    relevant_pixels = sum(len(pixel_distribution.get(angle, [])) for angle in range(86, 91))
    relevant_percentage = relevant_pixels / total_pixels if total_pixels > 0 else 0

    plt.figure(figsize=(5, 5), dpi=100)
    plt.bar(angles, normalized_distribution, width=1, color='red', edgecolor='red')
    plt.xlabel('Angle (degrees)', fontfamily='Arial')
    plt.ylabel('Pixels (Normalized)', fontfamily='Arial')
    plt.title('Pixel Distribution by Angle', fontfamily='Arial')
    plt.text(10, 0.1, f"86-91 degrees ratio: {relevant_percentage:.2%}", fontsize=10, fontfamily='Arial', color='black')
    plt.axvspan(86, 91, color='red', alpha=0.3, label='86-91 degrees')
    plt.legend()
    plt.savefig(save_path)
    plt.show()
    print(relevant_percentage)
    print(f"saved: {save_path}")

def save_angle_distribution(angle_counts, output_path):
    """save angle distribution to a text file"""
    with open(output_path, 'w') as f:
        f.write('Angle (degrees), Count\n')
        for angle, count in angle_counts.items():
            f.write(f'{angle}, {count}\n')
    print(f"angle saved: {output_path}")

In [ ]:
# === main===
def main():
    try:
        # 1. load
        binary_image = load_image(INPUT_IMAGE_PATH)

        # 2. detect lines using Hough transform
        lines = hough_transform(binary_image)
        
        # 3. calculate angle distribution and pixel distribution
        angle_counts, pixel_distribution = calculate_angle_distribution(lines)

        # 4. draw lines on a blank image
        combined_image = draw_lines(binary_image, lines)
        
        plt.imshow(combined_image, cmap='gray')
        plt.axis('off')
        plt.show()
        if cv2.imwrite(OUTPUT_IMAGE_PATH, combined_image):
            print(f"combined saved: {OUTPUT_IMAGE_PATH}")
        else:
            print(f"组合图像保存失败！")

        # 5. save angle distribution to a text file
        save_angle_distribution(angle_counts, OUTPUT_TXT_PATH)

        # 6. draw histogram of angle distribution
        save_histogram(angle_counts, pixel_distribution, HISTOGRAM_FILE_PATH)

    except Exception as e:
        print(f"error: {e}")

if __name__ == "__main__":
    main()